# Imports

In [36]:
from myosuite.simhive.myo_sim.test_sims import TestSims as loader
import scipy.sparse as spa
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
import numpy as np
import skvideo.io
import mujoco
import osqp

# Introduction and core function
In this tutorial a target trajectory will be replicated by MyoHand using MuJoCo inverse dynamics, i.e., given a sequence of joint angles *qpos*, a sequence of control *ctrl* will be generated. The following describes how the ***get_ctrl*** function works.
The equation to solve for *ctrl*, accordingly to muscle actuators dynamics (https://mujoco.readthedocs.io/en/latest/modeling.html#muscles), is:
$$
AM\cdot\left(gain\odot\left(act+timestep\cdot\frac{ctrl-act}{\tau}\right)+bias\right)-qfrc=0\tag{1}
$$
where
$$
\tau=\tau_D+(\tau_A-\tau_D)\cdot sigmoid\left(\frac{ctrl-act}{tausmooth}+0.5\right)
$$
To find a solution quickly, it is better to reformulate the equation and solve a quadratic program (QP), i.e.:
$$
\min_{x} \frac{1}{2}x^TPx+q^Tx~~~s.t.~~~lb\leq x\leq ub\tag{2}
$$
The major obstacle to this formulation is the sigmoid in the calculation of $\tau$. MuJoCo implements the sigmoid using the polynomial $6x^5-15x^4+10x^3$ clipped between 0 and 1. To solve the QP, the sigmoid is approximated here with $1.875x-0.4375$ and, to limit its range for a good approximation, *tausmooth* is set to 5. The target equation can then be rewritten as:
$$
AM\cdot\left(gain\odot\left(act+timestep\cdot\frac{ctrl-act}{(ctrl-act)\cdot \tau_1+\tau_2}\right)+bias\right)-qfrc=0\tag{3}
$$
where
$$
\tau_1=\frac{\tau_A-\tau_D}{tausmooth}\cdot 1.875,~~~\tau_2=(\tau_A+\tau_D)\cdot 0.5
$$
and consequently reformulated as:
$$
AM\cdot x+k=0\tag{4}
$$
where
$$
x=\left(timestep\cdot gain\odot\frac{ctrl-act}{(ctrl-act)\cdot \tau_1+\tau_2}\right),~~~k=AM\cdot(gain\odot act)+AM\cdot bias-qfrc
$$
Referring to equation $(2)$ then:
$$
P=2\cdot AM^T\cdot AM\tag{5}
$$
$$
q=2\cdot AM^T\cdot k\tag{6}
$$
$$
lb=timestep\cdot gain\odot\frac{1-act}{(1-act)\cdot \tau_1 + \tau_2}\tag{7}
$$
$$
ub=timestep\cdot gain\odot\frac{-act}{-act\cdot \tau_1 + \tau_2}\tag{8}
$$
After solving the QP for *x*, *ctrl* is then calculated as:
$$
ctrl = act + \frac{x\cdot\tau_2}{timestep\cdot gain-x\cdot\tau_1}\tag{9}
$$

In [ ]:
def solve_qp(P, q, lb, ub, x0):
    P = spa.csc_matrix(P)
    A = spa.csc_matrix(spa.eye(q.shape[0]))
    m = osqp.OSQP()
    m.setup(P=P, q=q, A=A, l=lb, u=ub, verbose=False)
    m.warm_start(x=x0)
    res = m.solve()
    return res.x

In [37]:
def get_ctrl(model, data, target_qpos):
    """
    Compute the control needed to reach the target position in the next mujoco step.
    """
    data_copy = deepcopy(data)
    # ---- qfrc computation
    data_copy.qpos = target_qpos.copy()
    data_copy.qvel = (target_qpos - data.qpos)/model.opt.timestep
    mujoco.mj_forward(model, data_copy)
    data_copy.qacc = 0
    mujoco.mj_inverse(model, data_copy)
    qfrc = data_copy.qfrc_inverse
    # ---- params computation
    gain = np.array([])
    bias = np.array([])
    for idx_actuator in range(model.nu):
        length = data_copy.actuator_length[idx_actuator]
        lengthrange = model.actuator_lengthrange[idx_actuator]
        velocity = data_copy.actuator_velocity[idx_actuator]
        acc0 = model.actuator_acc0[idx_actuator]
        prmb = model.actuator_biasprm[idx_actuator,:9]
        prmg = model.actuator_gainprm[idx_actuator,:9]
        bias = np.append(bias, mujoco.mju_muscleBias(length, lengthrange, acc0, prmb))
        gain = np.append(gain, min(-1, mujoco.mju_muscleGain(length, velocity, lengthrange, acc0, prmg)))
    tA = model.actuator_dynprm[:,0] * (0.5 + 1.5 * data.act)
    tD = model.actuator_dynprm[:,1] / (0.5 + 1.5 * data.act)
    tausmooth = model.actuator_dynprm[:,2]
    AM = data_copy.actuator_moment.T
    ts = model.opt.timestep
    act = data.act
    ctrl0 = data.ctrl
    # ---- ctrl computation
    t1 = (tA - tD) * 1.875 / tausmooth
    t2 = (tA + tD) * 0.5
    P = 2 * AM.T @ AM
    k = AM @ (gain * act) + AM @ bias - qfrc
    q = 2 * k @ AM
    lb = gain * (1 - act) * ts / (t2 + t1 * (1 - act))
    ub = - gain * act * ts / (t2 - t1 * act)
    x0 = (gain * (ctrl0 - act) * ts) / ((ctrl0 - act) * t1 + t2)
    x = solve_qp(P, q, lb, ub, x0)
    ctrl = act + x * t2 / (gain * ts - x * t1)
    return np.clip(ctrl,0,1)

# Main

In [45]:
# ---- initializations
model = loader.get_sim(None, 'hand/myohand.xml')
model.actuator_dynprm[:,2] = 5 # tausmooth
data = mujoco.MjData(model)
traj = pd.read_csv('data\\6_trajectory.csv').values

In [ ]:
# ---- main loop
all_ctrl = np.zeros(model.nu)
for idx in (pbar := tqdm(range(traj.shape[0]))):
    target_qpos = traj[idx, 1:]
    data.ctrl = get_ctrl(model, data, target_qpos)
    all_ctrl = np.vstack((all_ctrl, data.ctrl))
    mujoco.mj_step(model, data)

# Results

In [47]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
# ---- initializations
data_ref = mujoco.MjData(model) # data for reference trajectory
data_ach = mujoco.MjData(model) # data for achieved trajectory
# ---- camera settings
camera = mujoco.MjvCamera()
camera.azimuth = 166.553
camera.distance = 1.178
camera.elevation = -36.793
camera.lookat = np.array([-0.93762553, -0.34088276, 0.85067529])
options_ref = mujoco.MjvOption()
options_ref.flags[:] = 0
options_ref.geomgroup[1:] = 0
options_ach = mujoco.MjvOption()
options_ach.flags[:] = 0
options_ach.flags[4] = 1 # actuator ON
options_ach.geomgroup[1:] = 0
renderer = mujoco.Renderer(model)
renderer.scene.flags[:] = 0
# ---- generation loop
frames = []
for idx in (pbar := tqdm(range(traj.shape[0]))):
    # ---- reference trajectory
    data_ref.qpos = traj[idx, 1:]
    mujoco.mj_step(model, data_ref)
    renderer.update_scene(data_ref, camera=camera, scene_option=options_ref)
    pixels_ref = renderer.render()
    # ---- achieved trajectory
    data_ach.ctrl = all_ctrl[idx+1, :]
    mujoco.mj_step(model, data_ach)
    renderer.update_scene(data_ach, camera=camera, scene_option=options_ach)
    pixels_ach = renderer.render()
    # ---- merging 
    pixels = np.append(pixels_ref, pixels_ach, axis=1)
    frames.append(pixels)
# ---- writing
output_name = 'videos/myohand_freemovement.mp4'
skvideo.io.vwrite(output_name, np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})

Left, reference trajectory. Right, achieved trajectory by means of muscle activations resulting from inverse dynamics.

In [ ]:
show_video(output_name)